In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore')
from datetime import datetime

# 한글 폰트
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/HANBatang.ttf"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

sns.set(style='white', context='notebook', rc={'figure.figsize':(14, 10)})

C:\Users\JangSeongHyun\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\JangSeongHyun\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\JangSeongHyun\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
rc('font', family='Malgun Gothic')

In [4]:
data1 = pd.read_csv('data/data_pd.csv', low_memory=False)   # 유통사 구매 내역
data2 = pd.read_csv('data/data_ser.csv', index_col=0)       # 제휴사 서비스 이용 내역
demo = pd.read_csv('data/LPOINT_BIG_COMP_01_DEMO.csv')     # 고객 데모 정보

data1.shape, data2.shape, demo.shape

((3677256, 22), (228784, 16), (29913, 4))

In [10]:
data1.대분류명.nunique()

349

In [9]:
data1.중분류명.nunique()

60

In [12]:
data1.대분류명.nunique()

349

In [3]:
data1.columns = ['고객번호', '채널구분', '제휴사', '점포코드', '상품코드', '구매일자', '구매시간', '평균상품금액',
       '상품금액종류', '구매수량', '총구매금액', '영수증갯수', '엘포인트이용금액', '엘포인트횟수', '점포대분류코드',
       '점포중분류코드', '성별', '연령대', '거주지대분류코드', '소분류명', '중분류명', '대분류명']

demo.columns = ['고객번호', '성별', '연령대', '거주지대분류코드']

In [4]:
data1.head(2)

,고객번호,채널구분,제휴사,점포코드,상품코드,구매일자,구매시간,평균상품금액,상품금액종류,구매수량,...,엘포인트이용금액,엘포인트횟수,점포대분류코드,점포중분류코드,성별,연령대,거주지대분류코드,소분류명,대분류명,중분류명
0,M000034966,1,A01,A010045,PD0777,20210116,18,39000.0,2,2,...,NaN,NaN,Z16,Z16003,여성,40대,Z07,남성런닝/트레이닝화,스포츠패션,남성스포츠화
1,M000034966,1,A01,A010045,PD0796,20210116,18,100.0,1,1,...,NaN,NaN,Z16,Z16003,여성,40대,Z07,스포츠아대/헤어밴드,스포츠패션,스포츠잡화


In [5]:
data2.head(2)

,고객번호,제휴사,점포코드,채널구분,이용일자,방문일자,이용시간,총이용금액,서비스이용횟수,엘포인트이용금액,엘포인트횟수,점포대분류코드,점포중분류코드,성별,연령대,거주지대분류코드
0,M000034966,D01,D011011,1,20211030,20211030,9,3900,1,NaN,NaN,Z17,Z17005,여성,40대,Z07
1,M000059535,C01,C010085,2,20210303,20210305,17,46000,1,NaN,NaN,Z12,Z12019,여성,30대,Z12


In [6]:
demo.head(2)

,고객번호,성별,연령대,거주지대분류코드
0,M000034966,여성,40대,Z07
1,M000059535,여성,30대,Z12


## data1

### 기본적인 전처리 후 피처 만들기

#### num features

In [7]:
data1 = data1.sort_values(by=['구매일자', '구매시간']).reset_index(drop=True)
data1 = data1.fillna(0)

In [8]:
data1['월'] = data1['구매일자'].astype(str).str[4:6].astype(int)
data1['일'] = data1['구매일자'].astype(str).str[6:].astype(int)

# 구매일자, 구매시간 인코딩
def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

data1 = encode(data1, '월', 12)
data1 = encode(data1, '일', 31)
data1 = encode(data1, '구매시간', 23)

# 구매주기
data1['구매주기'] = data1.groupby('고객번호')['구매일자'].diff().fillna(0)

# 채널 구분 전처리(1: 오프라인, 0: 온라인)
data1['채널구분'] = 2 - data1['채널구분']

# 최근 구매
data1['구매일자시간'] = data1['구매일자'].astype(str) + data1['구매시간'].astype(str)

In [9]:
data1.iloc[:3, :15]

,고객번호,채널구분,제휴사,점포코드,상품코드,구매일자,구매시간,평균상품금액,상품금액종류,구매수량,총구매금액,영수증갯수,엘포인트이용금액,엘포인트횟수,점포대분류코드
0,M232396342,1,A04,A042226,PD0221,20210101,0,1500.0,1,1,1500.0,1,0.0,0.0,Z08
1,M232396342,1,A04,A042226,PD1344,20210101,0,2500.0,1,1,2500.0,1,0.0,0.0,Z08
2,M355489415,1,A04,A043757,PD0223,20210101,0,1000.0,1,2,2000.0,1,16800.0,1.0,Z10


In [10]:
data1.iloc[:3, 15:]

,점포중분류코드,성별,연령대,거주지대분류코드,소분류명,대분류명,중분류명,월,일,월_sin,월_cos,일_sin,일_cos,구매시간_sin,구매시간_cos,구매주기,구매일자시간
0,Z08011,여성,20대,Z08,하드캔디,과자,사탕/캔디,1,1,0.5,0.866025,0.201299,0.97953,0.0,1.0,0.0,202101010
1,Z08011,여성,20대,Z08,청주,주류,전통주,1,1,0.5,0.866025,0.201299,0.97953,0.0,1.0,0.0,202101010
2,Z10001,남성,20대,Z10,젤리,과자,사탕/캔디,1,1,0.5,0.866025,0.201299,0.97953,0.0,1.0,0.0,202101010


In [11]:
agg_dict = {
    # categorical
    '고객번호':['count'],
    '채널구분':['mean'],
    '제휴사':['nunique'],
    '점포코드':['nunique'],
    '상품코드':['nunique'],
    '점포대분류코드':['nunique'],
    '점포중분류코드':['nunique'],
    '구매일자':['nunique'],
    '구매일자시간':['nunique', 'max'],
    
    # numerical
    '구매주기':['mean', 'max', 'std'],
    '월_cos':['mean', 'std'],
    '월_sin':['mean', 'std'],
    '일_cos':['mean', 'std'],
    '일_sin':['mean', 'std'],
    '구매시간_cos':['mean', 'std'],
    '구매시간_sin':['mean', 'std'],
    '평균상품금액':['mean', 'std'],
    '총구매금액':['mean', 'std', 'sum'],
    '구매수량':['mean', 'std'],
    '엘포인트이용금액':['mean', 'std', 'sum']
}

In [12]:
f = data1.groupby('고객번호').agg(agg_dict)
f = f.fillna(0)
f.columns = [('_').join(column).upper() for column in f.columns]
f['엘포인트_금액비율'] = f['엘포인트이용금액_SUM'] / f['총구매금액_SUM']\

f = f.reset_index()

In [13]:
f = f.merge(demo, how='left', on='고객번호')
f.head()

,고객번호,고객번호_COUNT,채널구분_MEAN,제휴사_NUNIQUE,점포코드_NUNIQUE,상품코드_NUNIQUE,점포대분류코드_NUNIQUE,점포중분류코드_NUNIQUE,구매일자_NUNIQUE,구매일자시간_NUNIQUE,...,총구매금액_SUM,구매수량_MEAN,구매수량_STD,엘포인트이용금액_MEAN,엘포인트이용금액_STD,엘포인트이용금액_SUM,엘포인트_금액비율,성별,연령대,거주지대분류코드
0,M000034966,41,1.000000,3,4,37,2,3,7,8,...,654120.0,1.536585,0.809245,0.000000,0.000000,0.0,0.000000,여성,40대,Z07
1,M000136117,78,0.846154,3,4,59,3,4,29,42,...,27373630.0,1.461538,1.136094,24779.487179,143543.422417,1932800.0,0.070608,여성,30대,Z11
2,M000201112,20,1.000000,1,3,19,2,3,4,5,...,53120.0,1.500000,0.760886,0.000000,0.000000,0.0,0.000000,여성,50대,Z17
3,M000225114,133,1.000000,2,4,61,1,3,38,44,...,1937700.0,1.368421,0.722723,0.000000,0.000000,0.0,0.000000,여성,40대,Z17
4,M000261625,57,1.000000,1,4,33,2,4,22,31,...,6052700.0,1.245614,0.575720,0.000000,0.000000,0.0,0.000000,여성,40대,Z17


In [14]:
f.shape

(26917, 40)

In [15]:
f.iloc[:3, :20]

,고객번호,고객번호_COUNT,채널구분_MEAN,제휴사_NUNIQUE,점포코드_NUNIQUE,상품코드_NUNIQUE,점포대분류코드_NUNIQUE,점포중분류코드_NUNIQUE,구매일자_NUNIQUE,구매일자시간_NUNIQUE,구매일자시간_MAX,구매주기_MEAN,구매주기_MAX,구매주기_STD,월_COS_MEAN,월_COS_STD,월_SIN_MEAN,월_SIN_STD,일_COS_MEAN,일_COS_STD
0,M000034966,41,1.000000,3,4,37,2,3,7,8,2021122319,27.000000,304.0,78.249281,0.215369,0.476924,-0.399171,0.765950,-0.444105,0.581321
1,M000136117,78,0.846154,3,4,59,3,4,29,42,2021123020,14.435897,221.0,42.232686,0.133235,0.765450,0.081279,0.634349,0.191872,0.664448
2,M000201112,20,1.000000,1,3,19,2,3,4,5,2021112816,20.150000,315.0,71.713814,-0.528109,0.508973,-0.664711,0.186826,0.028156,0.622634


In [16]:
f.iloc[:3, 20:]

,일_SIN_MEAN,일_SIN_STD,구매시간_COS_MEAN,구매시간_COS_STD,구매시간_SIN_MEAN,구매시간_SIN_STD,평균상품금액_MEAN,평균상품금액_STD,총구매금액_MEAN,총구매금액_STD,총구매금액_SUM,구매수량_MEAN,구매수량_STD,엘포인트이용금액_MEAN,엘포인트이용금액_STD,엘포인트이용금액_SUM,엘포인트_금액비율,성별,연령대,거주지대분류코드
0,-0.639615,0.256068,-0.125837,0.498549,-0.839646,0.193877,11351.951220,16189.443508,15954.146341,22044.966430,654120.0,1.536585,0.809245,0.000000,0.000000,0.0,0.000000,여성,40대,Z07
1,0.315674,0.658225,-0.422015,0.357170,-0.745482,0.376890,275578.547009,486206.023173,350943.974359,599853.119652,27373630.0,1.461538,1.136094,24779.487179,143543.422417,1932800.0,0.070608,여성,30대,Z11
2,-0.192862,0.790551,0.111202,0.796176,-0.394200,0.492083,1843.275000,1622.217030,2656.000000,2165.353964,53120.0,1.500000,0.760886,0.000000,0.000000,0.0,0.000000,여성,50대,Z17


In [17]:
f['성별'].value_counts()

여성    18485
남성     8432
Name: 성별, dtype: int64

In [18]:
f['연령대'].value_counts()

40대    9217
30대    6791
50대    5325
20대    2864
60대    2043
70대     677
Name: 연령대, dtype: int64

In [19]:
f['거주지대분류코드'].value_counts()

Z10    7353
Z17    7066
Z16    2274
Z11    1702
Z04    1594
Z06    1025
Z14     835
Z13     816
Z12     721
Z08     654
Z09     579
Z15     570
Z05     531
Z03     530
Z01     419
Z07     173
Z02      75
Name: 거주지대분류코드, dtype: int64

#### RFM 
- Recency : 고객이 얼마나 최근에 구매하였는가? (최근성)
- Frequency : 고객이 얼마나 자주 방문했는가? (행동빈도)
- Monetary : 고객이 돈을 얼마나 썼는가? (구매금액)

In [20]:
rfm = f[['고객번호', '구매일자시간_MAX', '고객번호_COUNT', '총구매금액_SUM']]
rfm.columns = ['고객번호', 'Recency', 'Frequency', 'Monetary']

In [21]:
rfm.head()

,고객번호,Recency,Frequency,Monetary
0,M000034966,2021122319,41,654120.0
1,M000136117,2021123020,78,27373630.0
2,M000201112,2021112816,20,53120.0
3,M000225114,2021123119,133,1937700.0
4,M000261625,2021120413,57,6052700.0


In [22]:
# 날짜 데이터 전처리
rfm['Recency'] = pd.to_datetime(rfm['Recency'], format='%Y%m%d%H')

base_day = pd.to_datetime('2021010100', format='%Y%m%d%H')
time_diff = rfm['Recency'] - base_day                      ## 최근방문일과 기준 날짜의 시간 차이
time_in_seconds = [x.total_seconds() for x in time_diff]   ## 시간 차이를 초단위로 계산
rfm['Recency'] = time_in_seconds                           ## 변환된 데이터를 다시 삽입한다.

C:\Users\leeji\AppData\Local\Temp\ipykernel_13756\2417013435.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm['Recency'] = pd.to_datetime(rfm['Recency'], format='%Y%m%d%H')
C:\Users\leeji\AppData\Local\Temp\ipykernel_13756\2417013435.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm['Recency'] = time_in_seconds                           ## 변환된 데이터를 다시 삽입한다.


In [23]:
# code reference : https://zephyrus1111.tistory.com/13#c3
def get_score(level, data):
    '''
    Description :
    level안에 있는 원소를 기준으로
    1 ~ len(level)+ 1 까지 점수를 부여하는 함수
    
    Parameters :
    level = 튜플 또는 리스트 타입의 숫자형 데이터이며 반드시 오름차순으로 정렬되어 있어야함.
    예 - [1,2,3,4,5] O, [5,4,3,2,1] X, [1,3,2,10,4] X 
    data = 점수를 부여할 데이터. 순회가능한(iterable) 데이터 형식
    return :
    점수를 담고 있는 리스트 반환
    '''
    score = [] 
    for j in range(len(data)): 
        for i in range(len(level)): 
            if data[j] <= level[i]: 
                score.append(i+1) 
                break 
            elif data[j] > max(level): 
                score.append(len(level)+1) 
                break 
            else: 
                continue 
    return score
    
def get_rfm_grade(df, num_class, rfm_tick_point, rfm_col_map, suffix=None):
    '''
    Description :
    개별 고객에 대한 최근방문일/방문횟수/구매금액 데이터가 주어졌을때
    최근방문일/방문횟수/구매금액 점수를 계산하여 주어진 데이터 오른쪽에 붙여줍니다.
    
    Parameters :
    df = pandas.DataFrame 데이터
    num_class = 등급(점수) 개수
    rfm_tick_point = 최근방문일/방문횟수/구매금액에 대해서 등급을 나눌 기준이 되는 값
                    'quantile', 'min_max' 또는 리스트를 통하여 직접 값을 정할 수 있음.
                    단, 리스트 사용시 원소의 개수는 반드시 num_class - 1 이어야함.
                    quatile = 데이터의 분위수를 기준으로 점수를 매김
                    min_max = 데이터의 최소값과 최대값을 동일 간격으로 나누어 점수를 매김
    rfm_col_map = 최근방문일/방문횟수/구매금액에 대응하는 칼럼명
    예 - {'R':'Recency','F':'Frequency','M':'Monetary'}
    suffix = 최근방문일/방문횟수/구매금액에 대응하는 칼럼명 뒤에 붙는 접미사
    Return : 
    pandas.DataFrame
    '''
    ##### 필요모듈 체크
    import pandas as pd
    import numpy as np
    from sklearn import preprocessing
    
    ##### 파라미터 체크
    if not isinstance(df, pd.DataFrame): ## 데이터는 pd.DataFrame이어야 함.
        print('데이터는 pandas.DataFrame 객체여야 합니다.')
        return
    
    if isinstance(rfm_tick_point, dict) == False or isinstance(rfm_col_map, dict) == False: ## rfm_tick_point와 rfm_col_map은 모두 딕셔너리
        print(f'rfm_tick_point와 rfm_col_map은 모두 딕셔너리여야합니다.')
        return
    
    if len(rfm_col_map) != 3: ## rfm_col_map에는 반드시 3개의 키를 가져아함.
        print(f'rfm_col_map인자는 반드시 3개의 키를 가져야합니다. \n현재 rfm_col_map에는 {len(rfm_col_map)}개의 키가 있습니다.')
        return
    
    if len(rfm_tick_point) != 3: ## rfm_tick_point에는 반드시 3개의 키를 가져아함.
        print(f'rfm_tick_point인자는 반드시 3개의 키를 가져야합니다. \n현재 rfm_col_map에는 {len(rfm_col_map)}개의 키가 있습니다.')
        return
    
    if set(rfm_tick_point.keys()) != set(rfm_col_map.keys()): ## rfm_tick_point와 rfm_col_map은 같은 키를 가져야함.
        print(f'rfm_tick_point와 rfm_col_map은 같은 키를 가져야 합니다.')
        return
    
    if not set(rfm_col_map.values()).issubset(set(df.columns)):
        not_in_df = set(rfm_col_map.values())-set(df.columns)
        print(f'{not_in_df}이 데이터 칼럼에 있어야 합니다.')
        return
    
    for k, v in rfm_tick_point.items():
        if isinstance(v, str):
            if not v in ['quantile','min_max']:
                print(f'{k}의 값은 "quantile" 또는 "min_max"중에 하나여야 합니다.')
                return
        elif isinstance(v,list) or isinstance(v,tuple):
            if len(v) != num_class-1:
                print(f'{k}에 대응하는 리스트(튜플)의 원소는 {num_class-1}개여야 합니다.')
                return
    
    if suffix:
        if not isinstance(suffix, str):
            print('suffix인자는 문자열이어야합니다.')
            return
        
    ##### 최근방문일/방문횟수/구매금액 점수 부여
    for k, v in rfm_tick_point.items():
        if isinstance(v, str):
            if v == 'quantile':
                ## 데이터 변환
                scale = preprocessing.RobustScaler() ## 데이터의 범위 조작하기 쉽게 해주는 클래스 
                temp_data = np.array(df[rfm_col_map[k]]) ## 데이터를 Numpy 배열로 변환
                temp_data = temp_data.reshape((-1, 1)) ## scale을 적용하기위해 1차원 배열을 2차원으로 변환
                temp_data = scale.fit_transform(temp_data) ## 데이터를 평균은 0, 표준편차는 1을 갖도록 변환 
                temp_data = temp_data.squeeze() ## 데이터를 다시 1차원으로 변환
 
                ## 분위수 벡터
                quantiles_level = np.linspace(0, 1, num_class+1)[1:-1] ## 분위수를 구할 기준값을 지정 0과 1은 제외
                quantiles = [] ## 분위수를 담을 리스트
                for ql in quantiles_level:
                    quantiles.append(np.quantile(temp_data, ql)) ## 분위수를 계산하고 리스트에 삽입
            else: ## min_max인 경우
                ## 데이터 변환
                temp_data = np.array(df[rfm_col_map[k]])

                ## 등분점 계산
                quantiles = np.linspace(np.min(temp_data), np.max(temp_data), num_class+1)[1:-1] ## 최소값과 최대값을 점수 개수만큼 등간격으로 분할하는 점
        else: ## 직접 구분값을 넣어주는 경우
            temp_data = np.array(df[rfm_col_map[k]])
            quantiles = v ## 직접 구분값을 넣어줌
        score = get_score(quantiles, temp_data) ## 구분값을 기준으로 점수를 부여하고 리스트로 저장한다.
        new_col_name = rfm_col_map[k]+'_'+k ## 점수값을 담는 변수의 이름
        if suffix:
            new_col_name = rfm_col_map[k]+'_'+suffix
        df[new_col_name] = score ## 기존데이터 옆에 점수 데이터를 추가한다.
    return df

In [43]:
rfm_tick_point = {'R':'quantile', 'F':'quantile', 'M':'quantile'}
rfm_col_map = {'R':'Recency', 'F':'Frequency', 'M':'Monetary'}

result1 = get_rfm_grade(df=rfm, num_class=5, rfm_tick_point=rfm_tick_point, rfm_col_map=rfm_col_map)
result1

C:\Users\leeji\AppData\Local\Temp\ipykernel_13756\745985919.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_col_name] = score ## 기존데이터 옆에 점수 데이터를 추가한다.
C:\Users\leeji\AppData\Local\Temp\ipykernel_13756\745985919.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_col_name] = score ## 기존데이터 옆에 점수 데이터를 추가한다.
C:\Users\leeji\AppData\Local\Temp\ipykernel_13756\745985919.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

,고객번호,Recency,Frequency,Monetary,Recency_R,Frequency_F,Monetary_M
0,M000034966,30826800.0,41,654120.0,3,2,2
1,M000136117,31435200.0,78,27373630.0,5,3,5
2,M000201112,28656000.0,20,53120.0,2,2,1
3,M000225114,31518000.0,133,1937700.0,5,4,3
4,M000261625,29163600.0,57,6052700.0,2,3,5
...,...,...,...,...,...,...,...
26912,M999599111,27453600.0,29,1484022.0,2,2,3
26913,M999673157,30380400.0,21,2851370.0,3,2,4
26914,M999770689,31528800.0,277,1187140.0,5,5,3
26915,M999849895,28911600.0,62,1103792.0,2,3,3


In [32]:
rfm_tick_point={'R':'quantile', 'F':'quantile', 'M':'min_max'} ## 구매금액에 최소값 최대값을 이용한 방법 적용
rfm_col_map={'R':'Recency', 'F':'Frequency', 'M':'Monetary'}

result2 = get_rfm_grade(df=rfm, num_class=5, rfm_tick_point=rfm_tick_point, rfm_col_map=rfm_col_map)
result2

C:\Users\leeji\AppData\Local\Temp\ipykernel_13756\745985919.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_col_name] = score ## 기존데이터 옆에 점수 데이터를 추가한다.
C:\Users\leeji\AppData\Local\Temp\ipykernel_13756\745985919.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_col_name] = score ## 기존데이터 옆에 점수 데이터를 추가한다.


,고객번호,Recency,Frequency,Monetary,Recency_R,Frequency_F,Monetary_M
0,M000034966,30826800.0,41,654120.0,3,2,1
1,M000136117,31435200.0,78,27373630.0,5,3,1
2,M000201112,28656000.0,20,53120.0,2,2,1
3,M000225114,31518000.0,133,1937700.0,5,4,1
4,M000261625,29163600.0,57,6052700.0,2,3,1
...,...,...,...,...,...,...,...
26912,M999599111,27453600.0,29,1484022.0,2,2,1
26913,M999673157,30380400.0,21,2851370.0,3,2,1
26914,M999770689,31528800.0,277,1187140.0,5,5,1
26915,M999849895,28911600.0,62,1103792.0,2,3,1


#### 가중치 설정

In [44]:
## 필요 변수 추출
rfm_score = result1[['고객번호', 'Monetary', 'Recency_R', 'Frequency_F', 'Monetary_M']]

In [45]:
rfm_score

,고객번호,Monetary,Recency_R,Frequency_F,Monetary_M
0,M000034966,654120.0,3,2,2
1,M000136117,27373630.0,5,3,5
2,M000201112,53120.0,2,2,1
3,M000225114,1937700.0,5,4,3
4,M000261625,6052700.0,2,3,5
...,...,...,...,...,...
26912,M999599111,1484022.0,2,2,3
26913,M999673157,2851370.0,3,2,4
26914,M999770689,1187140.0,5,5,3
26915,M999849895,1103792.0,2,3,3


In [46]:
from tqdm import tqdm

In [47]:
def get_score(level, data, reverse=False):
    '''
    Description :
    level안에 있는 원소를 기준으로
    1 ~ len(level)+ 1 까지 점수를 부여하는 함수
    
    Parameters :
    level = 튜플 또는 리스트 타입의 숫자형 데이터이며 반드시 오름차순으로 정렬되어 있어야함.
    예 - [1,2,3,4,5] O, [5,4,3,2,1] X, [1,3,2,10,4] X 
    data = 점수를 부여할 데이터. 순회가능한(iterable) 데이터 형식
    reverse = 점수가 높을 때 그에 해당하는 값을 낮게 설정하고 싶을 때 True
    return :
    점수를 담고 있는 리스트 반환
    '''
    score = [] 
    for j in range(len(data)): 
        for i in range(len(level)): 
            if data[j] <= level[i]: 
                score.append(i+1) 
                break 
            elif data[j] > max(level): 
                score.append(len(level)+1) 
                break 
            else: 
                continue
    if reverse:
        return [len(level)+2-x for x in score]
    else:
        return score 

grid_number = 100 ## 눈금 개수, 너무 크게 잡으면 메모리 문제가 발생할 수 있음.
weights = []
for j in range(grid_number+1):
    weights += [(i/grid_number,j/grid_number,(grid_number-i-j)/grid_number)
                for i in range(grid_number+1-j)]
num_class = 5 ## 클래스 개수
class_level = np.linspace(1,5,num_class+1)[1:-1] ## 클래스를 나누는 지점을 정한다.
total_amount_of_sales = rfm_score['Monetary'].sum() ## 구매금액 총합 = 총 매출

In [48]:
max_std = 0 ## 표준편차 초기값
for w in tqdm(weights, position=0, desc='[Finding Optimal weights]'):
    ## 주어진 가중치에 따른 고객별 점수 계산
    score = w[0]*rfm_score['Recency_R'] + \
                        w[1]*rfm_score['Frequency_F'] + \
                        w[2]*rfm_score['Monetary_M'] 
    rfm_score['Class'] = get_score(class_level, score, True) ## 점수를 이용하여 고객별 등급 부여
    
    ## 등급별로 구매금액을 집계한다.
    grouped_rfm_score = rfm_score.groupby('Class')['Monetary'].sum().reset_index()
    
    ## 제약조건 추가 - 등급이 높은 고객들의 매출이 낮은 등급의 고객들보다 커야한다.
    grouped_rfm_score = grouped_rfm_score.sort_values('Class')
    
    temp_monetary = list(grouped_rfm_score['Monetary'])
    if temp_monetary != sorted(temp_monetary,reverse=True):
        continue
    
    ## 클래스별 구매금액을 총구매금액으로 나누어 클래스별 매출 기여도 계산
    grouped_rfm_score['Monetary'] = grouped_rfm_score['Monetary'].map(lambda x : x/total_amount_of_sales)
    std_sales = grouped_rfm_score['Monetary'].std() ## 매출 기여도의 표준편차 계산
    if max_std <= std_sales:
        max_std = std_sales ## 표준편차 최대값 업데이트
        optimal_weights = w  ## 가중치 업데이트

[Finding Optimal weights]:   0%|          | 0/5151 [00:00<?, ?it/s]C:\Users\leeji\AppData\Local\Temp\ipykernel_13756\3403313184.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm_score['Class'] = get_score(class_level, score, True) ## 점수를 이용하여 고객별 등급 부여
[Finding Optimal weights]:   0%|          | 1/5151 [00:00<39:02,  2.20it/s]C:\Users\leeji\AppData\Local\Temp\ipykernel_13756\3403313184.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm_score['Class'] = get_score(class_level, score, True) ## 점수를 이

In [49]:
print(optimal_weights)

(0.0, 0.26, 0.74)


In [50]:
score = optimal_weights[0]*rfm_score['Recency_R'] + \
        optimal_weights[1]*rfm_score['Frequency_F'] + \
        optimal_weights[2]*rfm_score['Monetary_M'] ## 고객별 점수 계산
 
rfm_score['Class'] = get_score(class_level, score, True) ## 고객별 등급 부여

C:\Users\leeji\AppData\Local\Temp\ipykernel_13756\3081312825.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm_score['Class'] = get_score(class_level, score, True) ## 고객별 등급 부여


In [51]:
## 클래스별 고객 수 계산
temp_rfm_score1 = rfm_score.groupby('Class')['고객번호'].count().reset_index().rename(columns={'CustomerID':'Count'})

## 클래스별 구매금액(매출)계산
temp_rfm_score2 = rfm_score.groupby('Class')['Monetary'].sum().reset_index()

## 클래스별 매출 기여도 계산
temp_rfm_score2['Monetary'] = temp_rfm_score2['Monetary'].map(lambda x : x/total_amount_of_sales)

## 데이터 결합
result_df = pd.merge(temp_rfm_score1, temp_rfm_score2, how='left', on=('Class'))

In [74]:
result_df

,Class,고객번호,Monetary
0,1,7282,0.780283
1,2,4072,0.109475
2,3,4474,0.064053
3,4,4686,0.033168
4,5,6403,0.013020


In [54]:
rfm_score = rfm_score.drop('Monetary', axis=1)

In [75]:
rfm_score

,고객번호,Recency_R,Frequency_F,Monetary_M,Class
0,M000034966,3,2,2,4
1,M000136117,5,3,5,1
2,M000201112,2,2,1,5
3,M000225114,5,4,3,3
4,M000261625,2,3,5,1
...,...,...,...,...,...
26912,M999599111,2,2,3,3
26913,M999673157,3,2,4,2
26914,M999770689,5,5,3,2
26915,M999849895,2,3,3,3


In [76]:
f = f.merge(rfm_score, how='left', on='고객번호')

In [77]:
f['연령대'] = f['연령대'].apply(lambda x : 'Z' if x == '20대'
                                else 'Y' if x in ['30대', '40대']
                                else 'X' if x == '50대'
                                else 'BB')

f['성별'] = f['성별'].apply(lambda x : 'M' if x == '남성'
                        else 'W')

f['label'] = f['성별'] + f['연령대']

In [82]:
f['label'].value_counts()

WY     10873
MY      5135
WX      3753
WZ      1971
WBB     1888
MX      1572
MZ       893
MBB      832
Name: label, dtype: int64

In [86]:
from sklearn.preprocessing import LabelEncoder

cat_features = ['성별', '연령대', '거주지대분류코드', 'label']

for cat in cat_features:
    le = LabelEncoder()
    f[cat] = le.fit_transform(f[cat])

In [182]:
ftr = f.drop(['고객번호', '성별', '연령대', '구매일자시간_MAX', 'label'], axis=1)
target = f[['label']]

In [183]:
ftr.shape, target.shape

((26917, 40), (26917, 1))

In [184]:
num_features = list(set(ftr.columns.tolist()) - set(['Recency_R', 'Frequency_F', 'Monetary_M', 'Class']))

In [185]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ftr[num_features] = ss.fit_transform(ftr[num_features])

In [186]:
ftr

,고객번호_COUNT,채널구분_MEAN,제휴사_NUNIQUE,점포코드_NUNIQUE,상품코드_NUNIQUE,점포대분류코드_NUNIQUE,점포중분류코드_NUNIQUE,구매일자_NUNIQUE,구매일자시간_NUNIQUE,구매주기_MEAN,...,구매수량_STD,엘포인트이용금액_MEAN,엘포인트이용금액_STD,엘포인트이용금액_SUM,엘포인트_금액비율,거주지대분류코드,Recency_R,Frequency_F,Monetary_M,Class
0,-0.507947,0.405074,0.390433,-0.209070,-0.425130,-0.047157,-0.147903,-0.753741,-0.718946,0.131323,...,-0.254129,-0.051921,-0.115566,-0.157449,-0.228244,-1.070321,3,2,2,4
1,-0.311387,-0.236089,0.390433,-0.209070,-0.032794,0.842966,0.258250,-0.088416,0.080137,-0.191906,...,-0.112900,0.118505,0.775261,0.155184,-0.141207,-0.175835,5,3,5,1
2,-0.619508,0.405074,-1.258211,-0.457384,-0.746132,-0.047157,-0.147903,-0.844467,-0.789454,-0.044903,...,-0.275025,-0.051921,-0.115566,-0.157449,-0.228244,1.165894,2,2,1,5
3,-0.019202,0.405074,-0.433889,-0.209070,0.002873,-0.937279,-0.147903,0.183762,0.127142,-0.383591,...,-0.291515,-0.051921,-0.115566,-0.157449,-0.228244,1.165894,5,4,3,3
4,-0.422948,0.405074,-1.258211,-0.209070,-0.496464,-0.047157,0.258250,-0.300110,-0.178390,-0.065912,...,-0.355034,-0.051921,-0.115566,-0.157449,-0.228244,1.165894,2,3,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26912,-0.571696,0.117656,1.214755,-0.209070,-0.674799,-0.047157,0.258250,-0.753741,-0.742449,0.308746,...,-0.306656,-0.051921,-0.115566,-0.157449,-0.228244,-1.741186,2,2,3,3
26913,-0.614195,0.405074,-0.433889,1.032500,-0.853133,-0.937279,0.258250,-0.572288,-0.577932,0.672801,...,-0.288242,-0.051921,-0.115566,-0.157449,-0.228244,-0.399457,3,2,4,2
26914,0.745789,0.149303,1.214755,2.522385,0.627044,-0.047157,1.882862,1.847073,1.795817,-0.460290,...,-0.270831,-0.014977,-0.019161,0.083226,1.316749,0.942273,5,5,3,2
26915,-0.396386,0.405074,0.390433,0.784186,-0.228962,0.842966,0.664403,-0.390836,-0.272400,-0.149592,...,1.439887,-0.051921,-0.115566,-0.157449,-0.228244,-1.741186,2,3,3,3


In [121]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from ngboost import NGBClassifier

In [155]:
from sklearn.model_selection import train_test_split

train_x, valid_x, train_y, valid_y = train_test_split(ftr, target, test_size=0.3, random_state=42)
train_x.shape, valid_x.shape

((18841, 40), (8076, 40))